In [1]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from jcopdl.callback import Callback, set_config

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [12]:
bs = 256
crop_size = 32

transform_data = transforms.Compose([
    transforms.CenterCrop(crop_size),
    transforms.ToTensor()
])

train_set = datasets.ImageFolder("cifar10/train/", transform=transform_data)
trainloader = DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=8)

test_set = datasets.ImageFolder("cifar10/test/", transform=transform_data)
testloader = DataLoader(test_set, batch_size=bs, shuffle=True, num_workers=8)

In [13]:
images, label = next(iter(trainloader))
images.shape

torch.Size([256, 3, 32, 32])

In [14]:
label2cat = train_set.classes
label2cat

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [15]:
from jcopdl.layers import linear_block

class CifarClf(nn.Module):
    def __init__(self, input_size, n1, n2, n3, output_size, drout=0.2):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Flatten(),
            linear_block(input_size, n1, dropout=drout),
            linear_block(n1, n2, dropout=drout),
            linear_block(n2, n3, dropout=drout),
            linear_block(n3, output_size, activation='lsoftmax')
        )
    
    def forward(self, x):
        return self.fc(x)

In [16]:
config = set_config({
    "input_size" : 3072,
    "n1" : 512,
    "n2" : 256,
    "n3" : 64,
    "output_size" : len(label2cat),
    "drout" : 0.2,
    "bs" : 256
})

In [17]:
model = CifarClf(config.input_size, config.n1, config.n2, config.n3, config.output_size, config.drout).to(device)
criterion = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0025)
callback = Callback(model, config, outdir="model_weight")

In [18]:
from tqdm.auto import tqdm

def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):
    if mode == "train":
        model.train()
    elif mode == "test":
        model.eval()
    cost = correct = 0
    for feature, target in tqdm(dataloader, desc=mode.title()):
        feature, target = feature.to(device), target.to(device)
        output = model(feature)
        loss = criterion(output, target)
        
        if mode == "train":
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        cost += loss.item() * feature.shape[0]
        correct += (output.argmax(1) == target).sum().item()
    cost = cost / len(dataset)
    acc = correct / len(dataset)
    return cost, acc

In [19]:
while True:
    train_cost, train_score = loop_fn("train", train_set, trainloader, model, criterion, optimizer, device)
    with torch.no_grad():
        test_cost, test_score = loop_fn("test", test_set, testloader, model, criterion, optimizer, device)
    
    # Logging
    callback.log(train_cost, test_cost, train_score, test_score)

    # Checkpoint
    callback.save_checkpoint()
        
    # Runtime Plotting
    callback.cost_runtime_plotting()
    callback.score_runtime_plotting()
    
    # Early Stopping
    if callback.early_stopping(model, monitor="test_score"):
        callback.plot_cost()
        callback.plot_score()
        break

RuntimeError: size mismatch, m1: [256 x 3072], m2: [1024 x 512] at C:/w/1/s/tmp_conda_3.7_055457/conda/conda-bld/pytorch_1565416617654/work/aten/src\THC/generic/THCTensorMathBlas.cu:273